In [2]:
###### Model Deployment in Sagemaker #########
##############################################
import boto3
from sagemaker.model import Model
from sagemaker import get_execution_role, Session 

role = get_execution_role()
session = Session()

ecr_image = '491085388405.dkr.ecr.us-east-1.amazonaws.com/fraud-ecr-1808:v1'

model = Model(
    image_uri=ecr_image,
    role=role,
    sagemaker_session=session 
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    endpoint_name='fraud-ml-endpoint'
)

----!

In [15]:
######## Inference using one row of Sample Data #########
#########################################################
import boto3
import json

# Create SageMaker runtime client
client = boto3.client('sagemaker-runtime', region_name='us-east-1')  # update region if needed

# Sample input — should match model's expected feature shape and order
payload = {
    "inputs": [[18.0, 1.0, 100987.0, 120870.0, 2.0, 2.0, 1.0, 1.0, 0.0, 0.0]]
}

# Invoke the deployed endpoint
response = client.invoke_endpoint(
    EndpointName='fraud-ml-endpoint',  # Make sure this matches exactly
    Body=json.dumps(payload),
    ContentType='application/json'
)

# Read and decode the prediction
result = response['Body'].read().decode('utf-8')
print("Prediction:", result)


Prediction: {"predictions":[0]}



In [16]:
######## Inference using two rows of Sample Data #########
##########################################################
import boto3
import json

# Create SageMaker runtime client
client = boto3.client('sagemaker-runtime', region_name='us-east-1')  # update region if needed

# Sample input — should match model's expected feature shape and order
# payload = {
#     "inputs": [[18.0, 1.0, 100987.0, 120870.0, 2.0, 2.0, 1.0, 1.0, 0.0, 0.0]]
# }

payload = {
    "inputs": [
        [18.0, 1.0, 100987.0, 120870.0, 2.0, 2.0, 1.0, 1.0, 0.0, 0.0],
        [25.0, 0.0, 200000.0, 150000.0, 1.0, 3.0, 0.0, 1.0, 0.0, 1.0]
    ]
}

# Invoke the deployed endpoint
response = client.invoke_endpoint(
    EndpointName='fraud-ml-endpoint',  # Make sure this matches exactly
    Body=json.dumps(payload),
    ContentType='application/json'
)

# Read and decode the prediction
result = response['Body'].read().decode('utf-8')
print("Prediction:", result)


Prediction: {"predictions":[0,0]}



In [ ]:
############ To create V2 of the End-point ##########
#####################################################

import boto3

# Initialize SageMaker client (region must match your ECR/SageMaker)
sm_client = boto3.client("sagemaker", region_name="us-east-1")

role = get_execution_role()


response = sm_client.create_model(
    ModelName="fraud-model-v2",
    PrimaryContainer={
        "Image": "491085388405.dkr.ecr.us-east-1.amazonaws.com/fraud-ecr-1808:v2"
    },
    ExecutionRoleArn=role
)
print("Model v2 Created:", response["ModelArn"]) 


In [11]:
###### To check what are the model names available in the Endpoint ########
###########################################################################
import boto3

sm_client = boto3.client("sagemaker", region_name="us-east-1")

response = sm_client.list_models(MaxResults=10)
for model in response["Models"]:
    print(model["ModelName"])


fraud-model-v2
fraud-ecr-1808-2025-08-18-06-57-15-444


In [12]:
########### A/B testing Configuration ##########
################################################

import boto3

sm_client = boto3.client("sagemaker")

endpoint_config_name = "fraud-endpoint-config-ab"

response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "Variant1",
            "ModelName": "fraud-model-v2",   # your v2 model
            "InitialInstanceCount": 1,
            "InstanceType": "ml.m5.large",
            "InitialVariantWeight": 0.7,   # 70% traffic
        },
        {
            "VariantName": "Variant2",
            "ModelName": "fraud-ecr-1808-2025-08-18-06-57-15-444",   # your ECR model
            "InitialInstanceCount": 1,
            "InstanceType": "ml.m5.large",
            "InitialVariantWeight": 0.3,   # 30% traffic
        },
    ],
)

print("Created Endpoint Config:", endpoint_config_name)


Created Endpoint Config: fraud-endpoint-config-ab


In [13]:
###### create or update the endpoint to use this new endpoint config (fraud-endpoint-config-ab)######
#####################################################################################################
import boto3

sm_client = boto3.client("sagemaker")

endpoint_name = "fraud-endpoint"

response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName="fraud-endpoint-config-ab"
)

print("✅ Endpoint created:", response['EndpointArn'])


✅ Endpoint created: arn:aws:sagemaker:us-east-1:491085388405:endpoint/fraud-endpoint


In [18]:
import boto3, json

runtime = boto3.client("sagemaker-runtime")

endpoint_name = "fraud-ml-endpoint"

# Example input with 10 variables
input_data = [
    [18.0, 1.0, 100987.0, 120870.0, 2.0, 2.0, 1.0, 1.0, 0.0, 0.0] 
]

payload = json.dumps({"instances": input_data})

response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=payload
)

print("Prediction:", response['Body'].read().decode("utf-8"))


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:14                                                                                   │
│                                                                                                  │
│   11                                                                                             │
│   12 payload = json.dumps({"instances": input_data})                                             │
│   13                                                                                             │
│ ❱ 14 response = runtime.invoke_endpoint(                                                         │
│   15 │   EndpointName=endpoint_name,                                                             │
│   16 │   ContentType="application/json",                                                         │
│   17 │   Body=payload                                                                            │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/client.py:601 in     │
│ _api_call                                                                                        │
│                                                                                                  │
│    598 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    599 │   │   │   │   )                                                                         │
│    600 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  601 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    602 │   │                                                                                     │
│    603 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    604                                                                                           │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/context.py:123 in    │
│ wrapper                                                                                          │
│                                                                                                  │
│   120 │   │   │   with start_as_current_context():                                               │
│   121 │   │   │   │   if hook:                                                                   │
│   122 │   │   │   │   │   hook()                                                                 │
│ ❱ 123 │   │   │   │   return func(*args, **kwargs)                                               │
│   124 │   │                                                                                      │
│   125 │   │   return wrapper                                                                     │
│   126                                                                                            │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/client.py:1074 in    │
│ _make_api_call                                                                                   │
│                                                                                                  │
│   1071 │   │   │   │   'error_code_override'                                                     │
│   1072 │   │   │   ) or error_info.get("Code")                                                   │
│   1073 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1074 │   │   │   raise error_class(parsed_response, opera

In [19]:
######## Inference using two rows of Sample Data #########
##########################################################
import boto3
import json

# Create SageMaker runtime client
client = boto3.client('sagemaker-runtime', region_name='us-east-1')  # update region if needed

# Sample input — should match model's expected feature shape and order
# payload = {
#     "inputs": [[18.0, 1.0, 100987.0, 120870.0, 2.0, 2.0, 1.0, 1.0, 0.0, 0.0]]
# }

payload = {
    "inputs": [
        [18.0, 1.0, 100987.0, 120870.0, 2.0, 2.0, 1.0, 1.0, 0.0, 0.0],
        [25.0, 0.0, 200000.0, 150000.0, 1.0, 3.0, 0.0, 1.0, 0.0, 1.0]
    ]
}

# Invoke the deployed endpoint
response = client.invoke_endpoint(
    EndpointName='fraud-ml-endpoint',  # Make sure this matches exactly
    Body=json.dumps(payload),
    ContentType='application/json'
)

# Read and decode the prediction
result = response['Body'].read().decode('utf-8')
print("Prediction:", result)


Prediction: {"predictions":[0,0]}

